# 协程

在解决并发问题的过程中，出现了进程、线程和协程。其中线程解决进程上下文切换带来的消耗，而协程是为了解决线程启动、管理和同步锁的开销。协程中以事件为单位，通过调度器来调度相应的事件。

协程执行有三种方法：
- await：程序阻塞在当前位置，进入被调用的协程函数，执行完毕返回后再继续。

In [7]:
import asyncio
async def crawl_page(url):
    print('crawling {}'.format(url))
    sleep_time=int(url.split('_')[-1])# 通过睡眠来模拟爬页面带来的开销
    await asyncio.sleep(sleep_time)
    print('OK {}'.format(url))

async def main(urls):
    for url in urls:
        await crawl_page(url)

%time asyncio.run(main(['url_1','url_2','url_3','url_4']))

RuntimeError: asyncio.run() cannot be called from a running event loop

- 通过`asyncio.create_task`创建事件。

In [ ]:
import asyncio
async def crawl_page(url):
    print('crawling {}'.format(url))
    sleep_time=int(url.split('_')[-1])# 通过睡眠来模拟爬页面带来的开销
    await asyncio.sleep(sleep_time)
    print('OK {}'.format(url))

async def main(urls):
    tasks=[asyncio.create_task(crawl_page(url)) for url in urls]
    for task in tasks:
        await task

%time asyncio.run(main(['url_1','url_2','url_3','url_4']))

- 通过`asyncio.gather()`启动事件。

In [ ]:
import asyncio
async def crawl_page(url):
    print('crawling {}'.format(url))
    sleep_time=int(url.split('_')[-1])# 通过睡眠来模拟爬页面带来的开销
    await asyncio.sleep(sleep_time)
    print('OK {}'.format(url))

async def main(urls):
    tasks=[asyncio.create_task(crawl_page(url)) for url in urls]
    await asyncio.gather(*tasks)

%time asyncio.run(main(['url_1','url_2','url_3','url_4']))

`asyncio.run()`启动任务，可以让其启动一个主程序的入口函数。

## 协程运行流程

- 顺序执行

In [10]:
import asyncio

async def work_1():
    print('work_1 start')
    await asyncio.sleep(1)
    print('work_1 end')

async def work_2():
    print('work_2 start')
    await asyncio.sleep(2)
    print('work_2 end')

async def main():
    print('before await')
    await work_1()
    print('awaited work_1')
    await work_2()
    print('awaited work_2')

%time asyncio.run(main())

c:\program files\python37\lib\site-packages\IPython\core\ultratb.py:1383: RuntimeWarning: coroutine 'main' was never awaited
  self.tb = tb


RuntimeError: asyncio.run() cannot be called from a running event loop

- 异步执行

In [16]:
import asyncio


async def work_1():
    # 4、work_1获得控制权
    print('work_1 start')
    # 5、交出了控制权，等待asyncio.sleep运行完毕
    await asyncio.sleep(1)
    # 8、asyncio.sleep 完成，work_1获得控制权
    print('work_1 end')


async def work_2():
    # 6、work_2获得控制权
    print('work_2 start')
    # 7、交出了控制权，等待asyncio.sleep运行完毕，没有可执行的事件，事件调度器进入等待
    await asyncio.sleep(2)
    # 11、asyncio.sleep 完成，work_2获得控制权
    print('work_2 end')


async def main():
    # 2、创建任务，并加入事件循环中。
    task1=asyncio.create_task(work_1())
    task2=asyncio.create_task(work_2())
    print('before await')
    # 3、main交出控制权，事件调度器调度task1执行，等待task1完成
    await task1
    # 9、task1完成，main获得控制权
    print('awaited work_1')
    # 10、等待task2完成
    await task2
    # 12、task2完成，main获得控制权
    print('awaited work_2')

# 1、程序进入main函数，启动事件循环。
%time asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop

总结：await有两个作用
- 交出控制权
- 等待被调用的协程函数执行完毕返回。

## 进阶

- 拦截协程异常:`asyncio.gather(*aws, loop=None, return_exceptions=True)`，返回的值中有异常信息。如果不用`return_exceptions`，则需要手动捕获异常。
- 取消任务：`task.cancel()`。

生产者、消费者模型

In [15]:
import asyncio
import random


async def consumer(queue, id):
    while True:
        val = await queue.get()
        print('consumer {} consume {}'.format(id, val))
        await asyncio.sleep(1)


async def producer(queue, id):
    for i in range(5):
        val = random.randint(1, 10)
        await queue.put(val)
        print('producer {} produce {}'.format(id, val))
        await asyncio.sleep(1)


async def main():
    queue = asyncio.Queue()
    consumer1 = asyncio.create_task(consumer(queue, 1))
    consumer2 = asyncio.create_task(consumer(queue, 2))
    producer3 = asyncio.create_task(producer(queue, 1))
    producer4 = asyncio.create_task(producer(queue, 2))

    await asyncio.sleep(15)
    consumer1.cancel()
    consumer2.cancel()
    await asyncio.gather(consumer1, consumer2, producer3, producer4, return_exceptions=True)

asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop

线程和协程的区别：协程是单线程的，由用户决定在哪里交出控制权，因此写协程时，必须在脑海中有清晰的事件循环概念，知道什么时候该暂停、什么时候等待I/O。

协程如何实现回调函数：通过task调用`add_done_callback`添加回调函数。